### Preprocessing
This module performs data transformation in preparation for the customer churn model training.

1. It reads raw data in CSV from GCS
2. Performs some basic transformations and
3. Persists to BigQuery

In [ ]:
spark

In [1]:
import sys
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from datetime import datetime
import random

In [2]:
# 1a. Arguments
pipelineID = random.randint(1, 10000)
projectNbr = "433578906282"
projectID = "vertex-ai-382806"
displayPrintStatements = True

In [3]:
# 1b. Variables 
bqDatasetNm = f"{projectID}.customer_churn_ds"
appBaseName = "customer-churn-model"
appNameSuffix = "preprocessing"
appName = f"{appBaseName}-{appNameSuffix}"
scratchBucketUri = f"s8s-spark-bucket-{projectNbr}/{appBaseName}/pipelineId-{pipelineID}/{appNameSuffix}"
sourceBucketUri = f"gs://s8s_data_bucket-{projectNbr}/customer_churn_train_data.csv"
bigQueryTargetTableFQN = f"{bqDatasetNm}.training_data"
pipelineExecutionDt = datetime.now().strftime("%Y%m%d%H%M%S")

In [4]:
# 1c. Display input and output
if displayPrintStatements:
    print("Starting preprocessing for the *Customer Churn* experiment")
    print(".....................................................")
    print(f"The datetime now is - {pipelineExecutionDt}")
    print(" ")
    print("INPUT PARAMETERS-")
    print(f"....pipelineID={pipelineID}")
    print(f"....projectID={projectID}")
    print(f"....projectNbr={projectNbr}")
    print(f"....displayPrintStatements={displayPrintStatements}")
    print(" ")
    print("EXPECTED SETUP-")  
    print(f"....BQ Dataset={bqDatasetNm}")
    print(f"....Source Data={sourceBucketUri}")
    print(f"....Scratch Bucket for BQ connector=gs://s8s-spark-bucket-{projectNbr}") 
    print("OUTPUT-")
    print(f"....BigQuery Table={bigQueryTargetTableFQN}")
    print(f"....Sample query-")
    print(f"....SELECT * FROM {bigQueryTargetTableFQN} WHERE pipeline_id='{pipelineID}' LIMIT 10" )
  

Starting preprocessing for the *Customer Churn* experiment
.....................................................
The datetime now is - 20240528132456
 
INPUT PARAMETERS-
....pipelineID=3828
....projectID=vertex-ai-382806
....projectNbr=433578906282
....displayPrintStatements=True
 
EXPECTED SETUP-
....BQ Dataset=vertex-ai-382806.customer_churn_ds
....Source Data=gs://s8s_data_bucket-433578906282/customer_churn_train_data.csv
....Scratch Bucket for BQ connector=gs://s8s-spark-bucket-433578906282
OUTPUT-
....BigQuery Table=vertex-ai-382806.customer_churn_ds.training_data
....Sample query-
....SELECT * FROM vertex-ai-382806.customer_churn_ds.training_data WHERE pipeline_id='3828' LIMIT 10


In [5]:
# 2. Spark Session creation
print('....Initializing spark & spark configs')
spark = SparkSession.builder.appName(appName).getOrCreate()

# Spark configuration setting for writes to BigQuery
spark.conf.set("parentProject", projectID)
spark.conf.set("temporaryGcsBucket", scratchBucketUri)

# Add Python modules
sc.addPyFile(f"gs://s8s_code_bucket-{projectNbr}/pyspark/common_utils.py")
import common_utils

....Initializing spark & spark configs


24/05/28 13:25:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
# 3. Read raw data in GCS into a Spark Dataframe
print('....Read source data')
rawChurnDF = spark.read.options(inferSchema = True, header= True).csv(sourceBucketUri)

....Read source data


In [7]:
# 4. View the data
if displayPrintStatements:
    print(rawChurnDF.count())
    rawChurnDF.show(2)

7043


+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+----------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|   PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+----------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|            No|         Yes|              No|         No|         No|    

In [8]:
# 5. Profile the data
if displayPrintStatements:
    rawChurnDF.describe().show()

24/05/28 13:25:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+----------+------+------------------+-------+----------+------------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+-----+
|summary|customerID|gender|     SeniorCitizen|Partner|Dependents|            tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|    MonthlyCharges|      TotalCharges|Churn|
+-------+----------+------+------------------+-------+----------+------------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+-----+
|  count|      7043|  7043|              7043|   7043|      7043|     

In [9]:
# 6. Check for spaces, nulls in monthly & total charges
print('....Exploratory Data Analysis')
if displayPrintStatements:
    rawChurnDF.createOrReplaceTempView("base_customer_churn")
    spark.sql("select count(*) from base_customer_churn where MonthlyCharges is null or MonthlyCharges=' '").show(5)
    spark.sql("select count(*) from base_customer_churn where TotalCharges is null or TotalCharges=' '").show(5)


....Exploratory Data Analysis
+--------+
|count(1)|
+--------+
|       0|
+--------+

+--------+
|count(1)|
+--------+
|      11|
+--------+



In [10]:
# 7. Replace spaces, space with null values in the TotalCharges and MonthlyCharges columns
print('....Replace space, nulls with None')
spaceReplacedDF = common_utils.fnReplaceSpaceWithNone(rawChurnDF)
if displayPrintStatements:
    print(spaceReplacedDF.count())

....Replace space, nulls with None
2024-05-28 13:26:12,450 - common_utils.py - INFO - ....Inside common_utils.fnReplaceSpaceWithNone
7043


In [11]:
# 8. Replace non-numeric values values in the TotalCharges and MonthlyCharges columns
print('....Replace non-numeric values in numeric columns with null')
nanReplacedDF = common_utils.fnReplaceNotANumberWithNone(spaceReplacedDF)
if displayPrintStatements:
    print(nanReplacedDF.count())

....Replace non-numeric values in numeric columns with null
2024-05-28 13:26:18,893 - common_utils.py - INFO - ....Inside common_utils.fnReplaceNotANumberWithNone
7043


In [12]:
# 9. Drop rows with null in columns
print('....Drop nulls')
nullDroppedDF = nanReplacedDF.na.drop()
if displayPrintStatements:
    print(nullDroppedDF.count())

....Drop nulls
7032


In [13]:
# 10. Replace 'No internet service' across columns to 'No'
print('....Replace -No internet service across columns- to -No-')
partiallyProcessedDF = common_utils.fnReplaceWithNoForInternetService(nullDroppedDF)
if displayPrintStatements:
    print(partiallyProcessedDF.count())

....Replace -No internet service across columns- to -No-
2024-05-28 13:27:37,160 - common_utils.py - INFO - ....Inside common_utils.fnReplaceWithNoForInternetService
7032


In [14]:
# 11. Add a bin/bucket category for tenure range using Spark SQL and write transformed to dataframe
print('....Add a bin for tenure')
modelTrainingReadyDF = common_utils.fnAddBinForTenure(partiallyProcessedDF, False, spark)
if displayPrintStatements:
    print(modelTrainingReadyDF.count())

....Add a bin for tenure
2024-05-28 13:27:41,927 - common_utils.py - INFO - ....Inside common_utils.fnAddBinForTenure


7032


In [15]:
# 12. Run summary statistics
if displayPrintStatements:
    modelTrainingReadyDF.describe().show()

+-------+----------+------+-------------------+-------+----------+------------------+------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+-----+
|summary|CustomerID|Gender|      SeniorCitizen|Partner|Dependents|            Tenure|Tenure_Group|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|    MonthlyCharges|      TotalCharges|Churn|
+-------+----------+------+-------------------+-------+----------+------------------+------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+-----+
|  count|      7032|  7032| 

In [16]:
# 13. Print schema
modelTrainingReadyDF.printSchema()

root
 |-- CustomerID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Tenure_Group: string (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: float (nullable = true)
 |-- TotalCharges: float (nullable = true)
 |-- Churn: string (nullable = true)



In [17]:
# 14. Format column names for consistency (title case to DB style & lowercase)
print('....Format column names for consistency')
persistDF = modelTrainingReadyDF.select("customerID", "gender", "SeniorCitizen", "Partner", "Dependents", "tenure", "Tenure_Group", "PhoneService", "MultipleLines", "InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod", "MonthlyCharges", "TotalCharges","Churn") \
                                .toDF("customer_id", "gender", "senior_citizen", "partner", "dependents", "tenure", "tenure_group", "phone_service", "multiple_lines", "internet_service", "online_security", "online_backup", "device_protection", "tech_support", "streaming_tv", "streaming_movies", "contract", "paperless_billing", "payment_method", "monthly_charges", "total_charges","churn") \
                                .withColumn("pipeline_id", lit(pipelineID).cast("string")) \
                                .withColumn("pipeline_execution_dt", lit(pipelineExecutionDt)) 

persistDF.printSchema()


....Format column names for consistency
root
 |-- customer_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- senior_citizen: integer (nullable = true)
 |-- partner: string (nullable = true)
 |-- dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- tenure_group: string (nullable = true)
 |-- phone_service: string (nullable = true)
 |-- multiple_lines: string (nullable = true)
 |-- internet_service: string (nullable = true)
 |-- online_security: string (nullable = true)
 |-- online_backup: string (nullable = true)
 |-- device_protection: string (nullable = true)
 |-- tech_support: string (nullable = true)
 |-- streaming_tv: string (nullable = true)
 |-- streaming_movies: string (nullable = true)
 |-- contract: string (nullable = true)
 |-- paperless_billing: string (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- monthly_charges: float (nullable = true)
 |-- total_charges: float (nullable = true)
 |-- churn: string (nu

In [18]:
# 15. Persist training dataset to a table in BQ with the pipeline ID and execution date for traceability
print('....Persist to BQ')  
persistDF.write.format('bigquery') \
.mode("append")\
.option('table', bigQueryTargetTableFQN) \
.save()

....Persist to BQ
